### Manipulation of Peaks de novo results of ETNP 2017 P2 samples LC-MS/MS data using python.

Starting with:

    Peaks de novo results (.csv) of PTM-optimized database searches

Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the running # (e.g., 233) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

We don't have technical duplicates here, sadly, unlike the MED4 Pro samples. I exported PeaksDN search results CSVs into my ETNP 2017 git repo:

In [1]:
cd /home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/

/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt


In [2]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

## 231: 100 m McLane pump filtered on 0.3 um GF-75

In [3]:
# read the CSV into a dataframe using the pandas read_csv function
peaks231_50 = pd.read_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN50.csv")

# also make a dataframe but keep only peptides >80% ALC
peaks231_80 = peaks231_50.loc[peaks231_50['ALC (%)'] >= 80].copy()

# how many de novo sequence candidates >50% ALC?
print("# redundant Peaks peptides >50% ALC in dataframe", len(peaks231_50))
# how many de novo sequence candidates >80ALC?
print("# redundant Peaks peptides >80% ALC in dataframe", len(peaks231_80))

#look at the dataframe
peaks231_50.head()

# redundant Peaks peptides >50% ALC in dataframe 5312
# redundant Peaks peptides >80% ALC in dataframe 1140


,Fraction,Scan,Source File,Peptide,Tag Length,ALC (%),length,m/z,z,RT,Area,Mass,ppm,PTM,local confidence (%),tag (>=0%),mode
0,1,14924,20170410_ETNP-231-100m-0.3um-JA2_01.raw,EN(+.98)LAALEK,8,98,8,444.7382,2,50.59,38500000.0,887.4600,2.0,Deamidation (NQ),98 99 100 99 98 99 100 96,EN(+.98)LAALEK,CID
1,1,11873,20170410_ETNP-231-100m-0.3um-JA2_01.raw,EN(+.98)N(+.98)LLAK,7,98,7,402.2114,2,41.36,7910000.0,802.4072,1.2,Deamidation (NQ),97 98 97 98 99 99 99,EN(+.98)N(+.98)LLAK,CID
2,1,16508,20170410_ETNP-231-100m-0.3um-JA2_01.raw,TGN(+.98)FLDPK,8,98,8,446.7253,2,55.16,5860000.0,891.4338,2.6,Deamidation (NQ),94 97 99 99 99 99 99 98,TGN(+.98)FLDPK,CID
3,1,27852,20170410_ETNP-231-100m-0.3um-JA2_01.raw,WLVNHPR,7,97,7,461.2513,2,81.49,1550000.0,920.4980,-10.8,NaN,99 100 100 96 97 97 98,WLVNHPR,CID
4,1,15671,20170410_ETNP-231-100m-0.3um-JA2_01.raw,TDENLPLGPK,10,97,10,542.2884,2,52.76,10900000.0,1082.5608,1.3,NaN,98 99 100 99 100 97 98 95 98 95,TDENLPLGPK,CID


In [8]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaks231_50['A'] = peaks231_50['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaks231_50['C'] = peaks231_50['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaks231_50['D'] = peaks231_50['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaks231_50['E'] = peaks231_50['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaks231_50['F'] = peaks231_50['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaks231_50['G'] = peaks231_50['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaks231_50['H'] = peaks231_50['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaks231_50 output, there will be no isoleucines (they're lumped in with leucines)
peaks231_50['I'] = peaks231_50['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaks231_50['K'] = peaks231_50['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaks231_50['L'] = peaks231_50['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaks231_50['M'] = peaks231_50['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaks231_50['N'] = peaks231_50['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaks231_50['P'] = peaks231_50['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaks231_50['Q'] = peaks231_50['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaks231_50['R'] = peaks231_50['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaks231_50['S'] = peaks231_50['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaks231_50['T'] = peaks231_50['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaks231_50['V'] = peaks231_50['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaks231_50['W'] = peaks231_50['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaks231_50['Y'] = peaks231_50['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaks231_50['c-carb'] = peaks231_50['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaks231_50['m-oxid'] = peaks231_50['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaks231_50['n-deam'] = peaks231_50['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of deamidated Q's in each peptide
peaks231_50['q-deam'] = peaks231_50['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# use a count function to enumerate the # of hydroxylated K's in each peptide
peaks231_50['k-hydr'] = peaks231_50['Peptide'].apply(lambda x: x.count('K(+15.99)'))

# use a count function to enumerate the # of hydroxylated P's in each peptide
peaks231_50['p-hydr'] = peaks231_50['Peptide'].apply(lambda x: x.count('P(+15.99)'))

# use a count function to enumerate the # of methylated R's in each peptide
peaks231_50['r-meth'] = peaks231_50['Peptide'].apply(lambda x: x.count('R(+14.02)'))

# create a column with 'stripped' peptide sequences using strip
peaks231_50['stripped peptide'] = peaks231_50['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaks231_50['stripped length'] = peaks231_50['stripped peptide'].apply(len)

# total the number of modifications in sequence
peaks231_50['ptm-total'] = peaks231_50['c-carb'] + peaks231_50['m-oxid'] + peaks231_50['n-deam'] + peaks231_50['q-deam'] + peaks231_50['k-hydr'] + peaks231_50['p-hydr'] + peaks231_50['r-meth']

# calculate NAAF numerator for each peptide k
peaks231_50['NAAF num.'] = peaks231_50['Area'] / peaks231_50['stripped length']

# write modified dataframe to new csv file
peaks231_50.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_PTMopt_DN50.csv")


# check out the results
peaks231_50.head()

,Fraction,Scan,Source File,Peptide,Tag Length,ALC (%),length,m/z,z,RT,...,q-deam,k-hydr,p-hydr,r-meth,stripped peptide,stripped length,ptm-total,NAAF num.,NAAF factor,L
0,1,14924,20170410_ETNP-231-100m-0.3um-JA2_01.raw,EN(+.98)LAALEK,8,98,8,444.7382,2,50.59,...,0,0,0,0,ENLAALEK,8,1,4.812500e+06,15.944261,2
1,1,11873,20170410_ETNP-231-100m-0.3um-JA2_01.raw,EN(+.98)N(+.98)LLAK,7,98,7,402.2114,2,41.36,...,0,0,0,0,ENLLAK,6,2,1.318333e+06,4.367761,2
2,1,16508,20170410_ETNP-231-100m-0.3um-JA2_01.raw,TGN(+.98)FLDPK,8,98,8,446.7253,2,55.16,...,0,0,0,0,TGNFLDPK,8,1,7.325000e+05,2.426841,1
3,1,27852,20170410_ETNP-231-100m-0.3um-JA2_01.raw,WLVNHPR,7,97,7,461.2513,2,81.49,...,0,0,0,0,WLVNHPR,7,0,2.214286e+05,0.733614,1
4,1,15671,20170410_ETNP-231-100m-0.3um-JA2_01.raw,TDENLPLGPK,10,97,10,542.2884,2,52.76,...,0,0,0,0,TDENLPLGPK,10,0,1.090000e+06,3.611272,2


In [19]:
# made a new dataframe that contains the sums of certain columns in the modified
# peptide dataframe above (for >50% ALC)

index = ['sample total']

data = {'A': peaks231_50['A'].sum(),
        'C': peaks231_50['C'].sum(),
        'D': peaks231_50['D'].sum(),
        'E': peaks231_50['E'].sum(),
        'F': peaks231_50['F'].sum(),
        'G': peaks231_50['G'].sum(),
        'H': peaks231_50['H'].sum(),
        'I': peaks231_50['I'].sum(),
        'K': peaks231_50['K'].sum(),
        'L': peaks231_50['L'].sum(),
        'M': peaks231_50['M'].sum(),
        'N': peaks231_50['N'].sum(),
        'P': peaks231_50['P'].sum(),
        'Q': peaks231_50['Q'].sum(),
        'R': peaks231_50['R'].sum(),
        'S': peaks231_50['S'].sum(),
        'T': peaks231_50['T'].sum(),
        'V': peaks231_50['V'].sum(),
        'W': peaks231_50['W'].sum(),
        'Y': peaks231_50['Y'].sum(),
        'c-carb': peaks231_50['c-carb'].sum(),
        'm-oxid': peaks231_50['m-oxid'].sum(),
        'n-deam': peaks231_50['n-deam'].sum(),
        'q-deam': peaks231_50['q-deam'].sum(),
        'k-hydr': peaks231_50['k-hydr'].sum(),
        'p-hydr': peaks231_50['p-hydr'].sum(),
        'r-meth': peaks231_50['r-meth'].sum(),
        'Total area': peaks231_50['Area'].sum(),
        'Total length': peaks231_50['stripped length'].sum()
       }

totalpeaks231_50 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', 'q-deam', 'k-hydr', 'p-hydr', 'r-meth', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaks231_50['% C w/ carb'] = totalpeaks231_50['c-carb'] / totalpeaks231_50['C'] 

# calculate percentage of M's that are oxidized
totalpeaks231_50['% M w/ oxid'] = totalpeaks231_50['m-oxid'] / totalpeaks231_50['M'] 

# calculate percentage of N's that are deamidated
totalpeaks231_50['% N w/ deam'] = totalpeaks231_50['n-deam'] / totalpeaks231_50['N'] 

# calculate percentage of Q's that are deamidated
totalpeaks231_50['% Q w/ deam'] = totalpeaks231_50['q-deam'] / totalpeaks231_50['Q'] 

# calculate percentage of K's that are hydroxylated
totalpeaks231_50['% K w/ hydr'] = totalpeaks231_50['k-hydr'] / totalpeaks231_50['K'] 

# calculate percentage of P's that are hydroxylated
totalpeaks231_50['% P w/ hydr'] = totalpeaks231_50['p-hydr'] / totalpeaks231_50['K'] 

# calculate percentage of R's that are methylated
totalpeaks231_50['% R w/ meth'] = totalpeaks231_50['r-meth'] / totalpeaks231_50['R'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaks231_50['NAAF denom.'] = totalpeaks231_50['Total area'] / totalpeaks231_50['Total length']

# write modified dataframe to new txt file
totalpeaks231_50.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_PTMopt_DN50_totals.csv")

totalpeaks231_50.head()

,A,C,D,E,F,G,H,I,K,L,...,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,% K w/ hydr,% P w/ hydr,% R w/ meth,NAAF denom.
sample total,3927,626,2010,2762,1551,2101,1048,0,5403,4784,...,1.264679e+10,41900,1.0,0.43424,0.202742,0.029278,0.135665,0.280585,0.287637,301832.734206


In [24]:
# use the calculated NAAF factor (in totalpeaks231 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

# don't have to worry here about DECOY hits messing with Area totals
# but we will with Comet results

NAAF50 = 301832.734206

# use NAAF >50% ALC to get NAAF
peaks231_50['NAAF factor'] = (peaks231_50['NAAF num.'])/NAAF50

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaks231_AA50 = peaks231_50[['stripped peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'K', 'I', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', 'q-deam', 'k-hydr', 'p-hydr', 'r-meth']].copy()

# multiply the NAAF50 factor by the AAs to normalize its abundance by peak area and peptide length

peaks231_AA50['A-NAAF50'] = peaks231_AA50['A'] * peaks231_50['NAAF factor']
peaks231_AA50['C-NAAF50'] = peaks231_AA50['C'] * peaks231_50['NAAF factor']
peaks231_AA50['D-NAAF50'] = peaks231_AA50['D'] * peaks231_50['NAAF factor']
peaks231_AA50['E-NAAF50'] = peaks231_AA50['E'] * peaks231_50['NAAF factor']
peaks231_AA50['F-NAAF50'] = peaks231_AA50['F'] * peaks231_50['NAAF factor']
peaks231_AA50['G-NAAF50'] = peaks231_AA50['G'] * peaks231_50['NAAF factor']
peaks231_AA50['H-NAAF50'] = peaks231_AA50['H'] * peaks231_50['NAAF factor']
peaks231_AA50['I-NAAF50'] = peaks231_AA50['I'] * peaks231_50['NAAF factor']
peaks231_AA50['K-NAAF50'] = peaks231_AA50['K'] * peaks231_50['NAAF factor']
peaks231_AA50['L-NAAF50'] = peaks231_AA50['L'] * peaks231_50['NAAF factor']
peaks231_AA50['M-NAAF50'] = peaks231_AA50['M'] * peaks231_50['NAAF factor']
peaks231_AA50['N-NAAF50'] = peaks231_AA50['N'] * peaks231_50['NAAF factor']
peaks231_AA50['P-NAAF50'] = peaks231_AA50['P'] * peaks231_50['NAAF factor']
peaks231_AA50['Q-NAAF50'] = peaks231_AA50['Q'] * peaks231_50['NAAF factor']
peaks231_AA50['R-NAAF50'] = peaks231_AA50['R'] * peaks231_50['NAAF factor']
peaks231_AA50['S-NAAF50'] = peaks231_AA50['S'] * peaks231_50['NAAF factor']
peaks231_AA50['T-NAAF50'] = peaks231_AA50['T'] * peaks231_50['NAAF factor']
peaks231_AA50['V-NAAF50'] = peaks231_AA50['V'] * peaks231_50['NAAF factor']
peaks231_AA50['W-NAAF50'] = peaks231_AA50['W'] * peaks231_50['NAAF factor']
peaks231_AA50['Y-NAAF50'] = peaks231_AA50['Y'] * peaks231_50['NAAF factor']

# multiply the NAAF50 factor by the PTMs normalize its abundance by peak area and peptide length

peaks231_AA50['ccarb-NAAF50'] = peaks231_AA50['c-carb'] * peaks231_AA50['NAAF factor']
peaks231_AA50['moxid-NAAF50'] = peaks231_AA50['m-oxid'] * peaks231_AA50['NAAF factor']
peaks231_AA50['ndeam-NAAF50'] = peaks231_AA50['n-deam'] * peaks231_AA50['NAAF factor']
peaks231_AA50['qdeam-NAAF50'] = peaks231_AA50['q-deam'] * peaks231_AA50['NAAF factor']
peaks231_AA50['khydr-NAAF50'] = peaks231_AA50['k-hydr'] * peaks231_AA50['NAAF factor']
peaks231_AA50['phydr-NAAF50'] = peaks231_AA50['p-hydr'] * peaks231_AA50['NAAF factor']
peaks231_AA50['rmeth-NAAF50'] = peaks231_AA50['r-meth'] * peaks231_AA50['NAAF factor']

# write the dataframe to a new csv
peaks231_AA50.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_PTMopt_DN50_AA_NAAF.csv")

peaks231_AA50.head()

,stripped peptide,NAAF factor,A,C,D,E,F,G,H,K,...,V-NAAF50,W-NAAF50,Y-NAAF50,ccarb-NAAF50,moxid-NAAF50,ndeam-NAAF50,qdeam-NAAF50,khydr-NAAF50,phydr-NAAF50,rmeth-NAAF50
0,ENLAALEK,15.944261,2,0,0,2,0,0,0,1,...,0.000000,0.000000,0.0,0.0,0.0,15.944261,0.0,0.0,0.0,0.0
1,ENLLAK,4.367761,1,0,0,1,0,0,0,1,...,0.000000,0.000000,0.0,0.0,0.0,8.735523,0.0,0.0,0.0,0.0
2,TGNFLDPK,2.426841,0,0,1,0,1,1,0,1,...,0.000000,0.000000,0.0,0.0,0.0,2.426841,0.0,0.0,0.0,0.0
3,WLVNHPR,0.733614,0,0,0,0,0,0,1,0,...,0.733614,0.733614,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,TDENLPLGPK,3.611272,0,0,1,1,0,1,0,1,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [25]:
# made a new dataframe that contains the sums of NAAF normalized AAs for comet231 PeaksDN results
# also contains the sums of the NAAF-corrected PTMs occurances for each affected residue

index = ['sample total']

data = {'NAAF': peaks231_AA50['NAAF factor'].sum(),
        'A-NAAF': peaks231_AA50['A-NAAF50'].sum(),
        'C-NAAF': peaks231_AA50['C-NAAF50'].sum(),
        'D-NAAF': peaks231_AA50['D-NAAF50'].sum(),
        'E-NAAF': peaks231_AA50['E-NAAF50'].sum(),
        'F-NAAF': peaks231_AA50['F-NAAF50'].sum(),
        'G-NAAF': peaks231_AA50['G-NAAF50'].sum(),
        'H-NAAF': peaks231_AA50['H-NAAF50'].sum(),
        'I-NAAF': peaks231_AA50['I-NAAF50'].sum(),
        'K-NAAF': peaks231_AA50['K-NAAF50'].sum(),
        'L-NAAF': peaks231_AA50['L-NAAF50'].sum(),
        'M-NAAF': peaks231_AA50['M-NAAF50'].sum(),
        'N-NAAF': peaks231_AA50['N-NAAF50'].sum(),
        'P-NAAF': peaks231_AA50['P-NAAF50'].sum(),
        'Q-NAAF': peaks231_AA50['Q-NAAF50'].sum(),
        'R-NAAF': peaks231_AA50['R-NAAF50'].sum(),
        'S-NAAF': peaks231_AA50['S-NAAF50'].sum(),
        'T-NAAF': peaks231_AA50['T-NAAF50'].sum(),
        'V-NAAF': peaks231_AA50['V-NAAF50'].sum(),
        'W-NAAF': peaks231_AA50['W-NAAF50'].sum(),
        'Y-NAAF': peaks231_AA50['Y-NAAF50'].sum(),
        'C-carb-NAAF': peaks231_AA50['ccarb-NAAF50'].sum(),
        'M-oxid-NAAF': peaks231_AA50['moxid-NAAF50'].sum(),
        'N-deam-NAAF': peaks231_AA50['ndeam-NAAF50'].sum(),
        'Q-deam-NAAF': peaks231_AA50['qdeam-NAAF50'].sum(),
        'K-hydr-NAAF': peaks231_AA50['khydr-NAAF50'].sum(),
        'P-hydr-NAAF': peaks231_AA50['phydr-NAAF50'].sum(),
        'R-meth-NAAF': peaks231_AA50['rmeth-NAAF50'].sum()
       }

totalpeaks231_AA50 = pd.DataFrame(data, columns=['NAAF', 'A-NAAF', 'C-NAAF', 'D-NAAF', 'E-NAAF', 'F-NAAF', \
                                                   'G-NAAF', 'H-NAAF', 'I-NAAF', 'K-NAAF', 'L-NAAF', 'M-NAAF', \
                                                   'N-NAAF', 'P-NAAF', 'Q-NAAF', 'R-NAAF', 'S-NAAF', \
                                                   'T-NAAF', 'V-NAAF', 'W-NAAF', 'Y-NAAF', 'C-carb-NAAF', \
                                                   'M-oxid-NAAF', 'N-deam-NAAF', 'Q-deam-NAAF', 'K-hydr-NAAF',\
                                                   'P-hydr-NAAF', 'R-meth-NAAF'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, P, and Rs


totalpeaks231_AA50['% C w/ carb. NAAF'] = totalpeaks231_AA50['C-carb-NAAF'] / totalpeaks231_AA50['C-NAAF']
totalpeaks231_AA50['% M w/ oxid. NAAF'] = totalpeaks231_AA50['M-oxid-NAAF'] / totalpeaks231_AA50['M-NAAF']
totalpeaks231_AA50['% N w/ deam. NAAF'] = totalpeaks231_AA50['N-deam-NAAF'] / totalpeaks231_AA50['N-NAAF']
totalpeaks231_AA50['% Q w/ deam. NAAF'] = totalpeaks231_AA50['Q-deam-NAAF'] / totalpeaks231_AA50['Q-NAAF']
totalpeaks231_AA50['% K w/ hydr. NAAF'] = totalpeaks231_AA50['K-hydr-NAAF'] / totalpeaks231_AA50['K-NAAF']
totalpeaks231_AA50['% P w/ hydr. NAAF'] = totalpeaks231_AA50['P-hydr-NAAF'] / totalpeaks231_AA50['P-NAAF']
totalpeaks231_AA50['% R w/ meth. NAAF'] = totalpeaks231_AA50['R-meth-NAAF'] / totalpeaks231_AA50['R-NAAF']

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaks231_AA50['NAAF check'] = totalpeaks231_AA50['NAAF'] / 301832.734206

# write modified dataframe to new txt file, same name + totals
totalpeaks231_AA50.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_PTMopt_DN50_NAAF_totals.csv")

totalpeaks231_AA50.head()

,NAAF,A-NAAF,C-NAAF,D-NAAF,E-NAAF,F-NAAF,G-NAAF,H-NAAF,I-NAAF,K-NAAF,...,P-hydr-NAAF,R-meth-NAAF,% C w/ carb. NAAF,% M w/ oxid. NAAF,% N w/ deam. NAAF,% Q w/ deam. NAAF,% K w/ hydr. NAAF,% P w/ hydr. NAAF,% R w/ meth. NAAF,NAAF check
sample total,5587.279526,5345.440129,473.34702,1669.296545,2564.015382,1083.482356,1842.142438,364.192253,0.0,3014.010158,...,877.284293,1589.339943,1.0,0.529633,0.441172,0.150038,0.144619,0.18273,0.307897,0.018511


### Same process but for de novo peptide >80 % ALC:

In [22]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaks231_80['A'] = peaks231_80['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaks231_80['C'] = peaks231_80['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaks231_80['D'] = peaks231_80['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaks231_80['E'] = peaks231_80['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaks231_80['F'] = peaks231_80['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaks231_80['G'] = peaks231_80['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaks231_80['H'] = peaks231_80['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaks231_80 output, there will be no isoleucines (they're lumped in with leucines)
peaks231_80['I'] = peaks231_80['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaks231_80['K'] = peaks231_80['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaks231_80['L'] = peaks231_80['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaks231_80['M'] = peaks231_80['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaks231_80['N'] = peaks231_80['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaks231_80['P'] = peaks231_80['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaks231_80['Q'] = peaks231_80['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaks231_80['R'] = peaks231_80['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaks231_80['S'] = peaks231_80['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaks231_80['T'] = peaks231_80['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaks231_80['V'] = peaks231_80['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaks231_80['W'] = peaks231_80['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaks231_80['Y'] = peaks231_80['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaks231_80['c-carb'] = peaks231_80['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaks231_80['m-oxid'] = peaks231_80['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaks231_80['n-deam'] = peaks231_80['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of deamidated Q's in each peptide
peaks231_80['q-deam'] = peaks231_80['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# use a count function to enumerate the # of hydroxylated K's in each peptide
peaks231_80['k-hydr'] = peaks231_80['Peptide'].apply(lambda x: x.count('K(+15.99)'))

# use a count function to enumerate the # of hydroxylated P's in each peptide
peaks231_80['p-hydr'] = peaks231_80['Peptide'].apply(lambda x: x.count('P(+15.99)'))

# use a count function to enumerate the # of methylated R's in each peptide
peaks231_80['r-meth'] = peaks231_80['Peptide'].apply(lambda x: x.count('R(+14.02)'))

# create a column with 'stripped' peptide sequences using strip
peaks231_80['stripped peptide'] = peaks231_80['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaks231_80['stripped length'] = peaks231_80['stripped peptide'].apply(len)

# total the number of modifications in sequence
peaks231_80['ptm-total'] = peaks231_80['c-carb'] + peaks231_80['m-oxid'] + peaks231_80['n-deam'] + peaks231_80['q-deam'] + peaks231_80['k-hydr'] + peaks231_80['p-hydr'] + peaks231_80['r-meth']

# calculate NAAF numerator for each peptide k
peaks231_80['NAAF num.'] = peaks231_80['Area'] / peaks231_80['stripped length']

# write modified dataframe to new csv file
peaks231_80.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_PTMopt_DN80.csv")

# check out the results
peaks231_80.head()

,Fraction,Scan,Source File,Peptide,Tag Length,ALC (%),length,m/z,z,RT,...,n-deam,q-deam,k-hydr,p-hydr,r-meth,stripped peptide,stripped length,ptm-total,NAAF num.,NAAF factor
0,1,14924,20170410_ETNP-231-100m-0.3um-JA2_01.raw,EN(+.98)LAALEK,8,98,8,444.7382,2,50.59,...,1,0,0,0,0,ENLAALEK,8,1,4.812500e+06,5.615403
1,1,11873,20170410_ETNP-231-100m-0.3um-JA2_01.raw,EN(+.98)N(+.98)LLAK,7,98,7,402.2114,2,41.36,...,2,0,0,0,0,ENLLAK,6,2,1.318333e+06,1.538280
2,1,16508,20170410_ETNP-231-100m-0.3um-JA2_01.raw,TGN(+.98)FLDPK,8,98,8,446.7253,2,55.16,...,1,0,0,0,0,TGNFLDPK,8,1,7.325000e+05,0.854708
3,1,27852,20170410_ETNP-231-100m-0.3um-JA2_01.raw,WLVNHPR,7,97,7,461.2513,2,81.49,...,0,0,0,0,0,WLVNHPR,7,0,2.214286e+05,0.258371
4,1,15671,20170410_ETNP-231-100m-0.3um-JA2_01.raw,TDENLPLGPK,10,97,10,542.2884,2,52.76,...,0,0,0,0,0,TDENLPLGPK,10,0,1.090000e+06,1.271852


In [17]:
# made a new dataframe that contains the sums of certain columns in modified 
#peptide dataframe above (for >80% ALC)

index = ['sample total']

data = {'A': peaks231_80['A'].sum(),
        'C': peaks231_80['C'].sum(),
        'D': peaks231_80['D'].sum(),
        'E': peaks231_80['E'].sum(),
        'F': peaks231_80['F'].sum(),
        'G': peaks231_80['G'].sum(),
        'H': peaks231_80['H'].sum(),
        'I': peaks231_80['I'].sum(),
        'K': peaks231_80['K'].sum(),
        'L': peaks231_80['L'].sum(),
        'M': peaks231_80['M'].sum(),
        'N': peaks231_80['N'].sum(),
        'P': peaks231_80['P'].sum(),
        'Q': peaks231_80['Q'].sum(),
        'R': peaks231_80['R'].sum(),
        'S': peaks231_80['S'].sum(),
        'T': peaks231_80['T'].sum(),
        'V': peaks231_80['V'].sum(),
        'W': peaks231_80['W'].sum(),
        'Y': peaks231_80['Y'].sum(),
        'c-carb': peaks231_80['c-carb'].sum(),
        'm-oxid': peaks231_80['m-oxid'].sum(),
        'n-deam': peaks231_80['n-deam'].sum(),
        'q-deam': peaks231_80['q-deam'].sum(),
        'k-hydr': peaks231_80['k-hydr'].sum(),
        'p-hydr': peaks231_80['p-hydr'].sum(),
        'r-meth': peaks231_80['r-meth'].sum(),
        'Total area': peaks231_80['Area'].sum(),
        'Total length': peaks231_80['stripped length'].sum()
       }

totalpeaks231_80 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', 'q-deam', 'k-hydr', 'p-hydr', 'r-meth', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaks231_80['% C w/ carb'] = totalpeaks231_80['c-carb'] / totalpeaks231_80['C'] 

# calculate percentage of M's that are oxidized
totalpeaks231_80['% M w/ oxid'] = totalpeaks231_80['m-oxid'] / totalpeaks231_80['M'] 

# calculate percentage of N's that are deamidated
totalpeaks231_80['% N w/ deam'] = totalpeaks231_80['n-deam'] / totalpeaks231_80['N'] 

# calculate percentage of Q's that are deamidated
totalpeaks231_80['% Q w/ deam'] = totalpeaks231_80['q-deam'] / totalpeaks231_80['Q'] 

# calculate percentage of K's that are hydroxylated
totalpeaks231_80['% K w/ hydr'] = totalpeaks231_80['k-hydr'] / totalpeaks231_80['K'] 

# calculate percentage of P's that are hydroxylated
totalpeaks231_80['% P w/ hydr'] = totalpeaks231_80['p-hydr'] / totalpeaks231_80['K'] 

# calculate percentage of R's that are methylated
totalpeaks231_80['% R w/ meth'] = totalpeaks231_80['r-meth'] / totalpeaks231_80['R'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaks231_80['NAAF denom.'] = totalpeaks231_80['Total area'] / totalpeaks231_80['Total length']

# write modified dataframe to new txt file
totalpeaks231_80.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_PTMopt_DN80_totals.csv")

totalpeaks231_80.head()

,A,C,D,E,F,G,H,I,K,L,...,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,% K w/ hydr,% P w/ hydr,% R w/ meth,NAAF denom.
sample total,941,42,454,746,350,486,81,0,836,1415,...,8.121100e+09,9476,1.0,0.60241,0.265203,0.091837,0.078947,0.223684,0.157969,857017.763508


In [18]:
# use the calculated NAAF factor (in totalpeaks231 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

# don't have to worry here about DECOY hits messing with Area totals
# but we will with Comet results

NAAF80 = 857017.763808

# use NAAF >80% ALC to get NAAF
peaks231_80['NAAF factor'] = (peaks231_80['NAAF num.'])/NAAF80

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaks231_AA80 = peaks231_80[['stripped peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', 'q-deam', 'k-hydr', 'p-hydr', 'r-meth']].copy()

# multiply the NAAF80 factor by the AAs to normalize its abundance by peak area and peptide length

peaks231_AA80['A-NAAF80'] = peaks231_AA80['A'] * peaks231_80['NAAF factor']
peaks231_AA80['C-NAAF80'] = peaks231_AA80['C'] * peaks231_80['NAAF factor']
peaks231_AA80['D-NAAF80'] = peaks231_AA80['D'] * peaks231_80['NAAF factor']
peaks231_AA80['E-NAAF80'] = peaks231_AA80['E'] * peaks231_80['NAAF factor']
peaks231_AA80['F-NAAF80'] = peaks231_AA80['F'] * peaks231_80['NAAF factor']
peaks231_AA80['G-NAAF80'] = peaks231_AA80['G'] * peaks231_80['NAAF factor']
peaks231_AA80['H-NAAF80'] = peaks231_AA80['H'] * peaks231_80['NAAF factor']
peaks231_AA80['I-NAAF80'] = peaks231_AA80['I'] * peaks231_80['NAAF factor']
peaks231_AA80['K-NAAF80'] = peaks231_AA80['K'] * peaks231_80['NAAF factor']
peaks231_AA80['L-NAAF80'] = peaks231_AA80['L'] * peaks231_80['NAAF factor']
peaks231_AA80['M-NAAF80'] = peaks231_AA80['M'] * peaks231_80['NAAF factor']
peaks231_AA80['N-NAAF80'] = peaks231_AA80['N'] * peaks231_80['NAAF factor']
peaks231_AA80['P-NAAF80'] = peaks231_AA80['P'] * peaks231_80['NAAF factor']
peaks231_AA80['Q-NAAF80'] = peaks231_AA80['Q'] * peaks231_80['NAAF factor']
peaks231_AA80['R-NAAF80'] = peaks231_AA80['R'] * peaks231_80['NAAF factor']
peaks231_AA80['S-NAAF80'] = peaks231_AA80['S'] * peaks231_80['NAAF factor']
peaks231_AA80['T-NAAF80'] = peaks231_AA80['T'] * peaks231_80['NAAF factor']
peaks231_AA80['V-NAAF80'] = peaks231_AA80['V'] * peaks231_80['NAAF factor']
peaks231_AA80['W-NAAF80'] = peaks231_AA80['W'] * peaks231_80['NAAF factor']
peaks231_AA80['Y-NAAF80'] = peaks231_AA80['Y'] * peaks231_80['NAAF factor']

# multiply the NAAF80 factor by the PTMs normalize its abundance by peak area and peptide length

peaks231_AA80['ccarb-NAAF80'] = peaks231_AA80['c-carb'] * peaks231_AA80['NAAF factor']
peaks231_AA80['moxid-NAAF80'] = peaks231_AA80['m-oxid'] * peaks231_AA80['NAAF factor']
peaks231_AA80['ndeam-NAAF80'] = peaks231_AA80['n-deam'] * peaks231_AA80['NAAF factor']
peaks231_AA80['qdeam-NAAF80'] = peaks231_AA80['q-deam'] * peaks231_AA80['NAAF factor']
peaks231_AA80['khydr-NAAF80'] = peaks231_AA80['k-hydr'] * peaks231_AA80['NAAF factor']
peaks231_AA80['phydr-NAAF80'] = peaks231_AA80['p-hydr'] * peaks231_AA80['NAAF factor']
peaks231_AA80['rmeth-NAAF80'] = peaks231_AA80['r-meth'] * peaks231_AA80['NAAF factor']

# write the dataframe to a new csv
peaks231_AA80.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_PTMopt_DN80_AA_NAAF.csv")

peaks231_AA80.head()

,stripped peptide,NAAF factor,A,C,D,E,F,G,H,I,...,V-NAAF80,W-NAAF80,Y-NAAF80,ccarb-NAAF80,moxid-NAAF80,ndeam-NAAF80,qdeam-NAAF80,khydr-NAAF80,phydr-NAAF80,rmeth-NAAF80
0,ENLAALEK,5.615403,2,0,0,2,0,0,0,0,...,0.000000,0.000000,0.0,0.0,0.0,5.615403,0.0,0.0,0.0,0.0
1,ENLLAK,1.538280,1,0,0,1,0,0,0,0,...,0.000000,0.000000,0.0,0.0,0.0,3.076560,0.0,0.0,0.0,0.0
2,TGNFLDPK,0.854708,0,0,1,0,1,1,0,0,...,0.000000,0.000000,0.0,0.0,0.0,0.854708,0.0,0.0,0.0,0.0
3,WLVNHPR,0.258371,0,0,0,0,0,0,1,0,...,0.258371,0.258371,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,TDENLPLGPK,1.271852,0,0,1,1,0,1,0,0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [23]:
# made a new dataframe that contains the sums of NAAF normalized AAs for comet231 PeaksDN results
# also contains the sums of the NAAF-corrected PTMs occurances for each affected residue

index = ['sample total']

data = {'NAAF': peaks231_AA80['NAAF factor'].sum(),
        'A-NAAF': peaks231_AA80['A-NAAF80'].sum(),
        'C-NAAF': peaks231_AA80['C-NAAF80'].sum(),
        'D-NAAF': peaks231_AA80['D-NAAF80'].sum(),
        'E-NAAF': peaks231_AA80['E-NAAF80'].sum(),
        'F-NAAF': peaks231_AA80['F-NAAF80'].sum(),
        'G-NAAF': peaks231_AA80['G-NAAF80'].sum(),
        'H-NAAF': peaks231_AA80['H-NAAF80'].sum(),
        'I-NAAF': peaks231_AA80['I-NAAF80'].sum(),
        'K-NAAF': peaks231_AA80['K-NAAF80'].sum(),
        'L-NAAF': peaks231_AA80['L-NAAF80'].sum(),
        'M-NAAF': peaks231_AA80['M-NAAF80'].sum(),
        'N-NAAF': peaks231_AA80['N-NAAF80'].sum(),
        'P-NAAF': peaks231_AA80['P-NAAF80'].sum(),
        'Q-NAAF': peaks231_AA80['Q-NAAF80'].sum(),
        'R-NAAF': peaks231_AA80['R-NAAF80'].sum(),
        'S-NAAF': peaks231_AA80['S-NAAF80'].sum(),
        'T-NAAF': peaks231_AA80['T-NAAF80'].sum(),
        'V-NAAF': peaks231_AA80['V-NAAF80'].sum(),
        'W-NAAF': peaks231_AA80['W-NAAF80'].sum(),
        'Y-NAAF': peaks231_AA80['Y-NAAF80'].sum(),
        'C-carb-NAAF': peaks231_AA80['ccarb-NAAF80'].sum(),
        'M-oxid-NAAF': peaks231_AA80['moxid-NAAF80'].sum(),
        'N-deam-NAAF': peaks231_AA80['ndeam-NAAF80'].sum(),
        'Q-deam-NAAF': peaks231_AA80['qdeam-NAAF80'].sum(),
        'K-hydr-NAAF': peaks231_AA80['khydr-NAAF80'].sum(),
        'P-hydr-NAAF': peaks231_AA80['phydr-NAAF80'].sum(),
        'R-meth-NAAF': peaks231_AA80['rmeth-NAAF80'].sum()
       }

totalpeaks231_AA80 = pd.DataFrame(data, columns=['NAAF', 'A-NAAF', 'C-NAAF', 'D-NAAF', 'E-NAAF', 'F-NAAF', \
                                                   'G-NAAF', 'H-NAAF', 'I-NAAF', 'K-NAAF', 'L-NAAF', 'M-NAAF', \
                                                   'N-NAAF', 'P-NAAF', 'Q-NAAF', 'R-NAAF', 'S-NAAF', \
                                                   'T-NAAF', 'V-NAAF', 'W-NAAF', 'Y-NAAF', 'C-carb-NAAF', \
                                                   'M-oxid-NAAF', 'N-deam-NAAF', 'Q-deam-NAAF', 'K-hydr-NAAF',\
                                                   'P-hydr-NAAF', 'R-meth-NAAF'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, P, and Rs


totalpeaks231_AA80['% C w/ carb. NAAF'] = totalpeaks231_AA80['C-carb-NAAF'] / totalpeaks231_AA80['C-NAAF']
totalpeaks231_AA80['% M w/ oxid. NAAF'] = totalpeaks231_AA80['M-oxid-NAAF'] / totalpeaks231_AA80['M-NAAF']
totalpeaks231_AA80['% N w/ deam. NAAF'] = totalpeaks231_AA80['N-deam-NAAF'] / totalpeaks231_AA80['N-NAAF']
totalpeaks231_AA80['% Q w/ deam. NAAF'] = totalpeaks231_AA80['Q-deam-NAAF'] / totalpeaks231_AA80['Q-NAAF']
totalpeaks231_AA80['% K w/ hydr. NAAF'] = totalpeaks231_AA80['K-hydr-NAAF'] / totalpeaks231_AA80['K-NAAF']
totalpeaks231_AA80['% P w/ hydr. NAAF'] = totalpeaks231_AA80['P-hydr-NAAF'] / totalpeaks231_AA80['P-NAAF']
totalpeaks231_AA80['% R w/ meth. NAAF'] = totalpeaks231_AA80['R-meth-NAAF'] / totalpeaks231_AA80['R-NAAF']

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaks231_AA80['NAAF check'] = totalpeaks231_AA80['NAAF'] / 857017.763808

# write modified dataframe to new txt file, same name + totals
totalpeaks231_AA80.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_PTMopt_DN80_NAAF_totals.csv")

totalpeaks231_AA80.head()

,NAAF,A-NAAF,C-NAAF,D-NAAF,E-NAAF,F-NAAF,G-NAAF,H-NAAF,I-NAAF,K-NAAF,...,P-hydr-NAAF,R-meth-NAAF,% C w/ carb. NAAF,% M w/ oxid. NAAF,% N w/ deam. NAAF,% Q w/ deam. NAAF,% K w/ hydr. NAAF,% P w/ hydr. NAAF,% R w/ meth. NAAF,NAAF check
sample total,1102.365059,1005.956125,55.790815,234.097585,439.842324,149.860168,376.208515,21.937118,0.0,329.469384,...,68.390865,110.256764,1.0,0.745412,0.426126,0.280357,0.137164,0.084814,0.118746,0.001286


### Visualizing the results

In [13]:
print("ALC max: ", peaks231_80['ALC (%)'].max())
print("ALC min: ", peaks231_80['ALC (%)'].min())

ALC max:  98
ALC min:  80


### Exporting txt files of stripped peptides at confidence cutoffs:

In [18]:
# keep only peptide column >50% ALC
pep50 = peaks231_50[["stripped peptide"]]

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

pep50.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN50_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 

!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN50_stripped_peptides.txt > /home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN50_stripped_peptides.fas

# look

print("# of DN peptide >50% ALC", len(pep50))
pep50.head()

# of DN peptide >50% ALC 5312


,stripped peptide
0,ENLAALEK
1,ENLLAK
2,TGNFLDPK
3,WLVNHPR
4,TDENLPLGPK


In [21]:
# keep only peptides  >80% ALC
peaks80 = peaks231_50.loc[peaks231_50['ALC (%)'] >= 80]

# see how many rows and double check
# peaks80.head(-10)

# keep only peptide column 
pep80 = peaks80[["stripped peptide"]]

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

pep80.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN80_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 

!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN80_stripped_peptides.txt > /home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN80_stripped_peptides.fas

print("# of DN peptide >80% ALC", len(pep80))
pep80.head()

# of DN peptide >80% ALC 1140


,stripped peptide
0,ENLAALEK
1,ENLLAK
2,TGNFLDPK
3,WLVNHPR
4,TDENLPLGPK


### Using BioPython to query peptide sequences

I installed the BioPython package using `pip install biopython`. All instructions and information [here](https://www.tutorialspoint.com/biopython/index.htm). 

GitHub project: https://github.com/biopython/biopython

I'm relying on the ProtParam module to parse sequences for relative AA composition, instability, secondary structure, instability, and hydrophobicity. You can read more about that module and the studies the indecies are derived from here:

https://biopython.org/wiki/ProtParam

In [ ]:
# Bio.SeqIO is the standard Sequence Input/Output interface for BioPython 1.43 and later
# Bio.SeqIO provides a simple uniform interface to input and output assorted sequence file formats.
# (including multiple sequence alignments), but will only deal with sequences as SeqRecord objects

# for accepted file formats see https://biopython.org/wiki/SeqIO

from Bio import SeqIO
#for seq_record in SeqIO.parse("/home/millieginty/Documents/git-repos/2017-etnp/data/MED4/MED2_tryp_1raw_db_peptides_nmod.fasta", "fasta"):
    #print(seq_record.id)
    #print(repr(seq_record.seq))
    #print(len(seq_record))
    
# I commented the print functions out so the output doesn't take up too much space. 

In [ ]:
# seeing what the ProtParam module can do with a single protein sequence:

from Bio.SeqUtils.ProtParam import ProteinAnalysis

test_seq = "MAEGEITTFTALTEKFNLPPGNYKKPKLLYCSNGGHFLRILPDGTVDGTRDRSDQHIQLQLSAESVGEVYIKSTETGQYLAMDTSGLLYGSQTPSEECLFLERLEENHYNTYTSKKHAEKNWFVGLKKNGSCKRGPRTHYGQKAILFLPLPV"

analysed_seq = ProteinAnalysis(test_seq)
print("molecular weight of seq =", analysed_seq.molecular_weight())

# calculates the aromaticity value of a protein according to Lobry & Gautier (1994, Nucleic Acids Res., 22, 3174-3180). 
# it's simply the relative frequency of Phe+Trp+Tyr.

analysed_seq.aromaticity()
print("aromaticity of seq =", analysed_seq.aromaticity())

# secondary_structure_fraction:
# this methods returns a list of the fraction of amino acids which tend to be in helix, turn or sheet. 
# AAs in helix: V, I, Y, F, W, L
# AAs in turn: N, P, G, S
# AAs in sheet: E, M, A, L
# the returned list contains 3 values: [Helix, Turn, Sheet]

analysed_seq.secondary_structure_fraction()
print("frac in H T S =", analysed_seq.secondary_structure_fraction())

# the instability index, an implementation of the method of Guruprasad et al. (1990, Protein Engineering, 4, 155-161).
# this method tests a protein for stability. 
# any value above 40 means the protein is unstable (=has a short half life)
# NOT SURE WHAT THIS MEANS FOR PEPTIDES, BUT WE COULD DO THIS FOR PROTEINS

analysed_seq.instability_index()
print("instability =", analysed_seq.instability_index())

# count_amino_acids will do just that, and get_amino_acids_percent will return %'s for each AA across the sequence. 
analysed_seq.get_amino_acids_percent()

# taking the returned dictionary and converting to a dataframe

aadict = analysed_seq.get_amino_acids_percent()
aadf = pd.DataFrame(list(aadict.items()),columns = ['residue','% occurance']) 

aadf.head()

In [ ]:
# use SeqIO and a loop to apply count_amino_acids to each sequence in the file
# aatot will give us the total number of each residue in the entire sample output

import collections
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis

all_aas = collections.defaultdict(int)
for record in SeqIO.parse("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN50_stripped_peptides.fas", "fasta"):
    x = ProteinAnalysis(str(record.seq))
    #print(record.id, x.count_amino_acids())
    for aa, count in x.count_amino_acids().items():
        all_aas[aa] += count        
        
# made a dataframe for amino acid total counts        
data = (all_aas)
aatot = pd.DataFrame(data, index = ['sample sequence total'])
aatot.head()

In [ ]:
from pandas import Series, DataFrame

with open('/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN50_stripped_peptides.fas') as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        lengths.append(len(seq_record.seq))
        
        
#converting lists to pandas Series    
s1 = Series(identifiers, name='ID')
s2 = Series(lengths, name='length')

#Gathering Series into a pandas DataFrame and rename index as ID column
idseq = DataFrame(dict(ID=s1, length=s2)).set_index(['ID'])

idseq.head()

In [ ]:
from pandas import Series, DataFrame



with open('/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN50_stripped_peptides.fas) as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    aa = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        lengths.append(len(seq_record.seq))
        aa.count_amino_acids(seq_record.seq)
        
        
#converting lists to pandas Series    
s1 = Series(identifiers, name='ID')
s2 = Series(lengths, name='length')
s3 = Series(aa, name='AAs')

#Gathering Series into a pandas DataFrame and rename index as ID column
idseq = DataFrame(dict(ID=s1, length=s2, AAs=s3)).set_index(['ID'])

idseq.head()

In [ ]:
from Bio import SeqIO
from Bio.SeqUtils import ProtParam

handle = open("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN50_stripped_peptides.fas") 
for record in SeqIO.parse(handle, "fasta"): 
    seq = str(record.seq)
    X = ProtParam.ProteinAnalysis(seq)
    print(X.count_amino_acids()) 
    #print X.get_amino_acids_percent() 
    #print X.molecular_weight() 
    #print X.aromaticity() 
    #print X.instability_index() 
    #print X.flexibility() 
    #print X.isoelectric_point() 
    #print X.secondary_structure_fraction()
    
# made a data series from the count_amino_acids function
# aacount = {X.count_amino_acids()}

# made a pandas dataframe from the series generated above
# aacount = pd.DataFrame(list(data.items()),columns = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']) 

aacount = pd.DataFrame(X.count_amino_acids(), index=[0])

# look at new dataframe

# aacount.head()